In [ ]:
import DirectDmTargets as dddm

In [ ]:
import wimprates as wr
wr.__version__

In [ ]:
wr

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numericalunits as nu

In [ ]:
from tqdm import tqdm

In [ ]:
from scipy.integrate import quad as scipy_int

In [ ]:
import pandas as pd

In [ ]:
import scipy

# Direct detection of Dark matter using different target materials #

Author:

Joran Angevaare <j.angevaare@nikef.nl>

Date:

14 october 2019 

## Goal ## 

- Roughly reproduce <https://journals.aps.org/prd/abstract/10.1103/PhysRevD.83.083505>
- Update the results thereof with more recent knowledge of the DM-distribution

### Approach ###
To achieve these goals, we must first get a decent recoil spectrum, that is flexible enough to have different astrophysical parameters. Further, it must also be flexible enough to be able to allow for different analytic functions to be added to the model. For instance, we must be able to incorporate the $\rm{SHM}^{++}$ as presented here <https://arxiv.org/abs/1810.11468>.

When we have a sufficiently flexible model, we want to add in the detector physics, we should therein incorporate at least the following parameters:
- target
  - material
  - cross-section
- threshold
- background
- cut efficiency  
- volume
- exposure

Finally we should perform the inference

## HALO model ##



In [ ]:
x = np.linspace(0,1000,200) * nu.km / nu.s
y2 = wr.observed_speed_dist(x, 0)
plt.plot(x/(nu.km / nu.s),y2)
# plt.axvline(310)
plt.xlabel("v [km/s]")
# plt.ylabel("$f (v)$")

In [ ]:
# NR's
energies = np.linspace(0.001, 100, 100)

# dr/dr
dr = wr.rate_wimp_std(energies, mw=50, sigma_nucleon=1e-45)

plt.plot(energies, dr, label="WIMPrates SHM")

plt.xlabel("Recoil energy [keV]")
plt.ylabel("Rate [events per (keV ton year)]")
plt.title("$m_\chi = 50$ GeV/c${}^2$, $\sigma_\chi = 10^{-45}$ cm${}^2$")
plt.xlim(0, energies.max())

use_SHM = dddm.SHM()
dr_shm = wr.rate_wimp_std(energies, mw=50, sigma_nucleon=1e-45, halo_model = use_SHM)
plt.plot(energies, dr_shm, label = "Modified SHM")
plt.yscale('log')
plt.legend()

## Detector smearing

# Inference #
Below we setup the inference

In [ ]:
# import emcee
# emcee.__version__

## Distribution of the DM ##
First we need to make a DM-rate spectrum

In [ ]:
dddm.GenSpectrum??

In [ ]:
def get_gal_spec(det):
    spec_class = dddm.GenSpectrum(50, 1e-45, use_SHM, dddm.experiment[det])
    spec_class.set_config(
        dict(E_max = 200,
             n_energy_bins = 200
            ))
    return spec_class.get_data(poisson = False)

In [ ]:
# plot_spectrum(xe_galact, label = 'un-smeared', linestyle = '--')
fig,ax=plt.subplots(figsize = (8,6))
dddm.plot_spectrum(get_gal_spec('Xe_simple'), 'red', label = 'Xe target ', plot_error = False)
dddm.plot_spectrum(get_gal_spec('Ge_simple'), 'green', label = 'Ge target', plot_error = False)
dddm.plot_spectrum(get_gal_spec('Ar_simple'), 'orange', label = 'Ar target', plot_error = False)
plt.yscale("log")
plt.xlabel("Recoil energy [keV]")
plt.ylabel("Rate [events per (keV ton year)]")
plt.legend()

In [ ]:
def get_det_spec(det, mw =50):
    spec_class = dddm.DetectorSpectrum(mw, 1e-45, use_SHM, dddm.experiment[det])
    spec_class.set_config(
        dict(E_max = 200,
             n_energy_bins = 200
            ))
    return spec_class.get_data(poisson = False)

In [ ]:
# plot_spectrum(xe_galact, label = 'un-smeared', linestyle = '--')
fig,ax=plt.subplots(figsize = (8,6))
dddm.plot_spectrum(get_det_spec('Xe_simple'), 'red', label = 'Xe det. resolution', plot_error = False)
dddm.plot_spectrum(get_det_spec('Ge_simple'), 'green', label = 'Ge det. resolution', plot_error = False)
dddm.plot_spectrum(get_det_spec('Ar_simple'), 'orange', label = 'Ar det. resolution', plot_error = False)
plt.yscale("log")
plt.ylim(1e-7,1e2)
plt.xlabel("Recoil energy [keV]")
plt.ylabel("Rate [events per (keV ton year)]")
plt.legend()

# Likelihood profiling

In [ ]:
dddm.plt_ll_sigma_det()

In [ ]:
dddm.plt_ll_mass_spec()
plt.ylim(-5000,0)

# Nested sampling

In [ ]:
fit_class = dddm.NestedSamplerStatModel('Xe_simple')
fit_class.config['sampler'] = 'nestle'
print(f"Fitting for parameters:\n{fit_class.config['fit_parameters']}")

In [ ]:
fit_class.run_nestle()
fit_class.get_summary()

In [ ]:
fit_class.show_corner()

In [ ]:
fit_unconstrained = dddm.NestedSamplerStatModel('Xe_simple')
fit_unconstrained.set_fit_parameters(fit_unconstrained.known_parameters)
print(f"Fitting for parameters:\n{fit_unconstrained.fit_class.config['fit_parameters']}")

In [ ]:
fit_unconstrained.run_nestle()
fit_unconstrained.get_summary()

In [ ]:
fit_unconstrained.show_corner()